<table>
<tr><td><img style="height: 150px;" src="images/geo_hydro1.jpg"></td>
<td bgcolor="#FFFFFF">
    <p style="font-size: xx-large; font-weight: 900; line-height: 100%">AG Dynamics of the Earth</p>
    <p style="font-size: large; color: rgba(0,0,0,0.5);">Jupyter notebooks</p>
    <p style="font-size: large; color: rgba(0,0,0,0.5);">Georg Kaufmann</p>
    </td>
</tr>
</table>

# Geophysikalisches Praktikum: `geodyn5` format

## Elevation data
----
*Georg Kaufmann,
Geophysics Section,
Institute of Geological Sciences,
Freie Universität Berlin,
Germany*

In [13]:
import numpy as np
import libGP

In this notebook, we reformat the DEM data for elevation from the Landesamt to our common `geodyn5`
format and plot the data.

----
## Reading DEM

The downloaded Lidar data for the FU Geocampus Berlin (`DGM1_388_5908.xyz`) looks like:

This simple ascii file contains triple of
- easting [m]
- northing [m]
- elevation [m]
without any meta data.

**Note:** This file contains more than 4 mill entries, therefore we have extracted the relevant elevation data
for the FU Geocampus, and stored them in the same format as `data/FUGeocampus/FUGeocampus_topo.xyz`.

We read this data file with `np.loadtxt` and create data array `data`:

In [26]:
data = np.loadtxt('data/FUGeocampus/FUGeocampus_topo.xyz',skiprows=6)
print(data.shape)

(160801, 3)


----
## Reformat to `geodyn5` format

The setup of the `geodyn5` format is:

- The first lines (starting with an exclamation mark) are **meta-data** of the dataset,
describing its content, technial details such as the source, the coordinate system:
    - !PROJECT - project name
    - !DATA:        - name of dtaa set
    - !OPERATOR: - operator names
    - !DATE:   - date of data set (creation)
    - !COORDINATES: - coordinates
    - !NOTES:       - note 

This list is not exclusive, you can assign your own meta-data, as long as they start with an exclamation mark.

- The last two rows of meta-data describe the structure of the following data set, the `geodyn5` dataset.
The column entries for the dtaa set (mandatory) are:
    - datetime - datetime string for data point [yyyy-mm-dd-hh:mm:ss]
    - easting  - easting coordinate [m]
    - northing - northing coordinate [m]
    - elev     - elevation [m]
    - depth    - sensor offset to ground [m]
    - field1   - field value1 [m,mGal,nT,...]
    - field2   - field value2 [m,mGal,nT,...]

- Then, the data lines start ...

To reformat these triples to the `geodyn5` format, we first create meta data as a string:

In [27]:
metaData = ''
metaData += "!PROJECT:     GP2024\n"
metaData += "!LOCATION:    FU Geocampus, Berlin\n"
metaData += "!DATA:        Lidar\n"
metaData += "!OPERATOR:\n"
metaData += "!DATE:\n"
metaData += "!COORDINATES: UTM 33T\n"
metaData += "!NOTES:       1m resolution\n"
metaData += "!VALUE:     datetime    easting    northing    elev    depth        field1        field2\n"
metaData += "!UNIT:                      [m]         [m]     [m]      [m]           [x]          [xx]"

print(metaData)

!PROJECT:     GP2024
!LOCATION:    FU Geocampus, Berlin
!DATA:        Lidar
!OPERATOR:
!DATE:
!COORDINATES: UTM 33T
!NOTES:       1m resolution
!VALUE:     datetime    easting    northing    elev    depth        field1        field2
!UNIT:                      [m]         [m]     [m]      [m]           [x]          [xx]


Next, we define a datetime string, then open the `geodyn5`data file, and save metadata and data to this file:

In [28]:
datetime = "2024-06-07-12:00:00"
elevFile = "data/FUGeocampus/2Dm_GP2024_DTM.xy"
f = open(elevFile,'w')
print(metaData,file=f)
# loop over elevation data
for i in range(data.shape[0]):
    print("%20s %12.2f %12.2f %8.2f %8.2f %8.2f %8.2f" % (datetime,data[i][0],data[i][1],data[i][2],0,0,0),file=f)
f.close()

This are the first lines of the newly created file:

In [34]:
!head -14 "data/FUGeocampus/2Dm_GP2024_DTM.xy"

!PROJECT:     GP2024
!LOCATION:    FU Geocampus, Berlin
!DATA:        Lidar
!OPERATOR:
!DATE:
!COORDINATES: UTM 33T
!NOTES:       1m resolution
!VALUE:     datetime    easting    northing    elev    depth        field1        field2
!UNIT:                      [m]         [m]     [m]      [m]           [x]          [xx]
 2024-06-07-12:00:00    388200.00   5809900.00    45.11     0.00     0.00     0.00
 2024-06-07-12:00:00    388201.00   5809900.00    45.02     0.00     0.00     0.00
 2024-06-07-12:00:00    388202.00   5809900.00    45.01     0.00     0.00     0.00
 2024-06-07-12:00:00    388203.00   5809900.00    45.00     0.00     0.00     0.00
 2024-06-07-12:00:00    388204.00   5809900.00    44.99     0.00     0.00     0.00


... done